# Лабораторная работа 2. Ранжирование (11 баллов)

Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете так же должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Сдавать задание после указанного срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).

## Поисковое ранжирование

![](http://i.imgur.com/2QnD2nF.jpg)

Задачу поискового ранжирования можно описать следующим образом: имеется множество документов $d \in D$ и множество запросов $q \in Q$. Требуется оценить *степень релевантности* документа по отношению к запросу: $(q, d) \mapsto r$, относительно которой будет производиться ранжирование. Для восстановления этой зависимости используются методы машинного обучения. Обычно используется три типа:
 - признаки запроса $q$, например: мешок слов текста запроса, его длина, ...
 - документа $d$, например: значение PageRank, мешок слов, доменное имя, ...
 - пары $(q, d)$, например: число вхождений фразы из запроса $q$ в документе $d$, ...

Одна из отличительных особенностей задачи ранжирования от классических задач машинного обучения заключается в том, что качество результата зависит не от предсказанных оценок релевантности, а от порядка следования документов в рамках конкретного запроса, т.е. важно не абсолютное значение релевантности (его достаточно трудно формализовать в виде числа), а то, более или менее релевантен документ, относительно других документов.
### Подходы к решению задачи ранжирования
Существуют 3 основных подхода, различие между которыми в используемой функции потерь:
  
1. **Pointwise подход**. В этом случае рассматривается *один объект* (в случае поискового ранжирования - конкретный документ) и функция потерь считается только по нему. Любой стандартный классификатор или регрессор может решать pointwise задачу ранжирования, обучившись предсказывать значение таргета. Итоговое ранжирование получается после сортировки документов к одному запросу по предсказанию такой модели.
2. **Pairwise подход**. В рамках данной модели функция потерь вычисляется по *паре объектов*. Другими словами, функция потерь штрафует модель, если отражированная этой моделью пара документов оказалась в неправильном порядке.
3. **Listwise подход**. Этот подход использует все объекты для вычисления функции потерь, стараясь явно оптимизировать правильный порядок.

### Оценка качества

Для оценивания качества ранжирования найденных документов в поиске используются асессорские оценки. Само оценивание происходит на скрытых от обучения запросах $Queries$. Для этого традиционно используется метрика *DCG* ([Discounted Cumulative Gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)) и ее нормализованный вариант — *nDCG*, всегда принимающий значения от 0 до 1.
Для одного запроса DCG считается следующим образом:
$$ DCG = \sum_{i=1}^P\frac{(2^{rel_i} - 1)}{\log_2(i+1)}, $$

где $P$ — число документов в поисковой выдаче, $rel_i$ — релевантность (асессорская оценка) документа, находящегося на i-той позиции.

*IDCG* — идеальное (наибольшее из возможных) значение *DCG*, может быть получено путем ранжирования документов по убыванию асессорских оценок.

Итоговая формула для расчета *nDCG*:

$$nDCG = \frac{DCG}{IDCG} \in [0, 1].$$

Чтобы оценить значение *nDCG* на выборке $Queries$ ($nDCG_{Queries}$) размера $N$, необходимо усреднить значение *nDCG* по всем запросам  выборки:
$$nDCG_{Queries} = \frac{1}{N}\sum_{q \in Queries}nDCG(q).$$

Пример реализации метрик ранжирование на python можно найти [здесь](https://gist.github.com/mblondel/7337391).

Загрузите данные конкурса [Интернет-математика 2009](http://imat2009.yandex.ru/datasets). Там же находится описание данных. Разбейте обучающую выборку на обучение и контроль в соотношении 70 / 30. Обратите внимание на формат данных: разбивать необходимо множество запросов, а не строчки датасета.

In [39]:
import numpy as np
import pandas as pd

In [14]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, GroupKFold, GroupShuffleSplit, cross_val_predict

In [54]:
# read dataset, extract query ids
X, y = datasets.load_svmlight_file('./imat2009-datasets/imat2009_learning.txt')
with open('./imat2009-datasets/imat2009_learning.txt') as file:
    query_ids = np.array([int(line.split('#')[1].strip()) for line in file.readlines()])

In [55]:
relevant_queries = (pd.Series(y, index=query_ids).groupby(level=0).sum() > 0)
relevant_queries = relevant_queries[relevant_queries].index
query_mask = np.where([q in relevant_queries for q in query_ids])[0]
X, y, query_ids = X[query_mask], y[query_mask], query_ids[query_mask]

In [56]:
gss = GroupShuffleSplit(1, test_size=0.3)

train_ids, test_ids = next(gss.split(X, y, query_ids))

In [57]:
X_train, y_train = X[train_ids], y[train_ids]
X_test, y_test = X[test_ids], y[test_ids]

In [58]:
X_train.shape, X_test.shape

((64483, 245), (27699, 245))

Далее рассмотрим несколько подходов предсказания релевантности. Для оценивания качества моделей используйте метрику nDCG на контроле. В случае подбора гиперпараметров используйте кросс-валидацию по 5 блокам, где разбиение должно быть по запросам, а не строчкам датасета.

**(1.3 балл) Задание 1.** *Pointwise* подход. Воспользовавшись известными вам техниками построения линейной регрессии, обучите модель, предсказывающую оценку асессора.

In [59]:
import metrics as m

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [61]:
def ndcg(y_true, y_pred, groups):
    return np.nanmean([m.ndcg_score(np.array(y_true)[np.array(groups) == g], 
                                 np.array(y_pred)[np.array(groups) == g]) for g in set(groups)])

In [62]:
def cv_test(regr):
    cv_pred = cross_val_predict(regr, X_train, y_train, groups=query_ids[train_ids], cv=5)
    cv_ndcg = ndcg(y_train, cv_pred, query_ids[train_ids])
    regr.fit(X_train, y_train)
    val_pred = regr.predict(X_test)
    val_ndcg = ndcg(y_test, val_pred, query_ids[test_ids])
    return dict(
        cv_pred = cv_pred,
        cv_ndcg = cv_ndcg,
        val_pred = val_pred,
        val_ndcg = val_ndcg,
    )

In [64]:
cv_res_lin = cv_test(LinearRegression())
print(cv_res_lin['val_ndcg'], cv_res_lin['cv_ndcg'])

0.8417875026337917 0.8358849037424148


###  Ранжируем с XGBoost

XGBoost имеет несколько функций потерь для решения задачи ранжирования:
1. **reg:linear** — данную функцию потерь можно использовать для решения задачи ранжирование *pointwise* подходом.
2. **rank:pairwise** — в качестве *pairwise* модели в XGBoost реализован [RankNet](http://icml.cc/2015/wp-content/uploads/2015/06/icml_ranking.pdf), в котором минимизируется гладкий функционал качества ранжирования: $$ Obj = \sum_{i \prec j} \mathcal{L}\left(a(x_j) - a(x_i)\right) \rightarrow min $$ $$ \mathcal{L}(M) = log(1 + e^{-M}), $$ где $ a(x) $ - функция ранжирования. Суммирование ведется по всем парам объектов, для которых определено отношение порядка, например, для пар документов, показанных по одному запросу. Таким образом функция потерь штрафует за то, что пара объектов неправильно упорядочена.
3. **rank:map, rank:ndcg** — реализация [LambdaRank](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf) для двух метрик: [MAP](https://en.wikipedia.org/wiki/Information_retrieval#Mean_average_precision) и **nDCG**. Известно, что для того, чтобы оптимизировать негладкий функционал, такой как **nDCG**,  нужно домножить градиент функционала $ Obj(a) $ на значение $\Delta NDCG_{ij} $ — изменение значения функционала качества при замене $x_i$ на $ x_j$.  Поскольку для вычисления метрик необходимы все объекты выборки, то эти две ранжирующие функции потерь являются представителями класса *listwise* моделей.

**(2.7 балла) Задание 2.** Обучите модели **reg:linear**, **rank:pairwise** и **rank:ndcg**, в качестве метрики оценки качества (*eval_metric*) используя *nDCG*, а в качестве бустера решающее дерево. Настройте [параметры](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md) алгоритма и добейтесь высокого качества на тестовой выборке.

In [65]:
import xgboost as xgb

In [67]:
cv_res_reg_lin = cv_test(xgb.XGBRegressor(objective='reg:linear'))
print(cv_res_reg_lin['val_ndcg'], cv_res_reg_lin['cv_ndcg'])

0.853174921068299 0.8505660788941586


In [68]:
cv_res_rank_pair = cv_test(xgb.XGBModel(objective='rank:pairwise', groups=query_ids[train_ids]))
print(cv_res_rank_pair['val_ndcg'], cv_res_rank_pair['cv_ndcg'])

0.8485820474612767 0.8441353403123042


In [70]:
cv_res_rank_ndcg = cv_test(xgb.XGBModel(objective='rank:ndcg', groups=query_ids[train_ids]))
print(cv_res_rank_ndcg['val_ndcg'], cv_res_rank_ndcg['cv_ndcg'])

0.7541392592596167 0.7499498002147456


nDCG для rank:ndcg выглядит неплохо, однако алгоритм выдал по 0.5 всем документам..
возможно, это как-то связано с [issue](https://github.com/dmlc/xgboost/issues/2768)

In [91]:
np.ptp(cv_res_rank_ndcg['val_ndcg']), np.ptp(cv_res_rank_ndcg['cv_ndcg'])

(0.0, 0.0)

#### Пользовательская функция потерь

Библиотека XGBoost позволяет использовать пользовательские функции потерь. Для этого необходимо реализовать функцию, принимающую на вход вектор предсказанных значений и обучающую выборку, и возвращающую градиент и гессиан, посчитанный по входным данным.

Важно отметить, что XGBoost использует диагональную аппроксимацию гессиана, таким образом все недиагональные элементы считаются малозначимыми и приравниваются нулю, поэтому и градиент, и гессиан являются векторами длины размера обучающей выборки.

**(5.7 балла) Задание 3.** Реализуйте экспоненциальную функцию потерь для XGBoost:
$$ Obj = \sum_{i \prec j} \mathcal{L}\left(a(x_j) - a(x_i)\right) \rightarrow min $$ $$ \mathcal{L}(M) = e^{-M} $$

Обучите модель с помощью данной функции потерь, настройте параметры.

**Комментарии к реализации**

В случае ранжирования XGBoost'у необходимо знать о разбиении всех объектов на группы. В нашем случае в одну группу будут входить документы, соответствующие одному запросу. Функция, считающая градиент и гессиан по данным, должна знать данное разбиение датасета. Однако питоновский интерфейс класса *DMatrix* (в котором хранится датасет) не дает возможности получить это разбиение. В этом случае нужно реализовать функцию потерь в качестве функтора, конструктор которого принимает разбиение на группы в качестве параметра.

Пример реализации своей функции потерь можно найти на соответствующем семинаре.

In [71]:
class ExponentialPairwiseLoss(object):
    def __init__(self, groups):
        self.groups = np.array(groups)
                            
    def __call__(self, y_true, preds):
        y_true = np.array(y_true)
        preds = np.array(preds)
#         print(preds.min(), preds.max())
        
        grad = np.zeros_like(y_true)
        hess = np.zeros_like(y_true)
        for group in set(self.groups):
            group_indices = np.where(self.groups == group)[0]
            for j in group_indices:
                for i in group_indices:
                    if y_true[i] < y_true[j]:
#                         print(i, j, y_true[i] , y_true[j], preds[j] - preds[i])
                        exp = np.exp(-(preds[j] - preds[i]))
                        grad[i] += exp
                        grad[j] += - exp
                        hess[i] += exp
                        hess[j] += exp
        return grad, hess

In [72]:
from sklearn.base import clone

def cross_val_predict_with_obj(regr, obj, X_train, y_train, groups, cv_n=5):
    kf = GroupKFold(cv_n)
    cv_preds = np.zeros_like(y_train)
    for tr_i, te_i in kf.split(X_train, y_train, groups):
        regr = clone(regr).set_params(objective=obj(groups[tr_i]))
        regr.fit(X_train[tr_i], y_train[tr_i])
        cv_preds[te_i] = regr.predict(X_train[te_i])
    return cv_preds

In [77]:
cv_pred_exp = cross_val_predict_with_obj(xgb.XGBModel(), 
                                         ExponentialPairwiseLoss, X_train, y_train, query_ids[train_ids])

In [80]:
regr = xgb.XGBModel(objective=ExponentialPairwiseLoss(groups=query_ids[train_ids]))
regr.fit(X_train, y_train)
pred_exp = regr.predict(X_test)

In [81]:
cv_res_rank_exp = dict(
    cv_pred = cv_pred_exp,
    cv_ndcg = ndcg(y_train, cv_pred_exp, query_ids[train_ids]),
    val_pred = pred_exp,
    val_ndcg = ndcg(y_test, pred_exp, query_ids[test_ids])
)

In [82]:
print(cv_res_rank_exp['val_ndcg'], cv_res_rank_exp['cv_ndcg'])

0.8495593036691309 0.8464663564304656


**(1.3 балл) Задание 4.** Сравните построенные модели с точки зрения метрики nDCG на контроле и проанализируйте полученные результаты:
  - какая модель работает лучше всего для данной задачи? 
  - в чем достоинства/недостатки каждой? 
  - сравните модели между собой: 
   - получается ли сравнимое качество линейного pointwise подхода с остальными моделями? 
   - заметна ли разница в качестве при использовании бустинга с разными функциями потерь?

Лучше всего для задачи отработал xgboost pointwise подход. Особенной разницы в результатах не заметно. 